<a href="https://colab.research.google.com/github/dmswl0707/CoronaCoprosperityCard_DataScience/blob/main/Card_Payment_Ratio_by_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import json

from google.colab import drive
drive.mount('/content/drive') #구글 드라이브에 접근

filename1='/content/drive/My Drive/Colab Notebooks/data/card_2020.csv' #파일 경로설정
data1=pd.read_csv(filename1) #csv 파일 불러오기
print(data1.head())
print(data1.info()) #파일정보

Mounted at /content/drive
   receipt_dttm  adstrd_code  ... selng_cascnt   salamt
0      20200104   1174066000  ...          463  5843230
1      20200104   1174066000  ...           33  7835550
2      20200104   1174066000  ...           53  4589800
3      20200104   1174066000  ...          339  9267240
4      20200104   1174066000  ...           19  4441000

[5 rows x 7 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3713952 entries, 0 to 3713951
Data columns (total 7 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   receipt_dttm          int64 
 1   adstrd_code           int64 
 2   adstrd_nm             object
 3   mrhst_induty_cl_code  int64 
 4   mrhst_induty_cl_nm    object
 5   selng_cascnt          int64 
 6   salamt                int64 
dtypes: int64(5), object(2)
memory usage: 198.3+ MB
None


In [3]:
filename2='/content/drive/My Drive/Colab Notebooks/data/index.csv' #파일 경로설정
data2=pd.read_csv(filename2) #csv 파일 불러오기
print(data2.head())
print(data2.info()) #파일정보

   period     catl    catm age gender sido sigungu         cgi
0  201901  건강/의료용품  건강관리용품  20      F   서울     관악구  115.374642
1  201901  건강/의료용품  건강관리용품  20      F   서울     광진구  119.596454
2  201901  건강/의료용품  건강관리용품  20      F   서울     도봉구  156.992847
3  201901  건강/의료용품  건강관리용품  20      F   서울     동작구   58.342726
4  201901  건강/의료용품  건강관리용품  20      F   서울     마포구  145.147595
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127525 entries, 0 to 127524
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   period   127525 non-null  int64  
 1   catl     127525 non-null  object 
 2   catm     127525 non-null  object 
 3   age      127525 non-null  object 
 4   gender   127525 non-null  object 
 5   sido     127525 non-null  object 
 6   sigungu  127525 non-null  object 
 7   cgi      127525 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 7.8+ MB
None


In [4]:
np.sum(pd.isnull(data1)) #결측치 확인
np.sum(pd.isnull(data2))

period     0
catl       0
catm       0
age        0
gender     0
sido       0
sigungu    0
cgi        0
dtype: int64

In [5]:
###필요한 index 추출
 
data1_sample=data1[['mrhst_induty_cl_nm','selng_cascnt','salamt']]
print(data1_sample.head(100))

data2_sample=data2[['age','catl','catm']]
print(data2_sample.head(100))


   mrhst_induty_cl_nm  selng_cascnt    salamt
0                  약국           463   5843230
1                치과의원            33   7835550
2                 한의원            53   4589800
3                  의원           339   9267240
4               자동차정비            19   4441000
..                ...           ...       ...
95                 스넥           296   4850800
96                 주점           329  12114900
97               서양음식          1097  10263165
98               중국음식           184   2510600
99               일식회집           120   5480600

[100 rows x 3 columns]
   age     catl    catm
0   20  건강/의료용품  건강관리용품
1   20  건강/의료용품  건강관리용품
2   20  건강/의료용품  건강관리용품
3   20  건강/의료용품  건강관리용품
4   20  건강/의료용품  건강관리용품
..  ..      ...     ...
95  20       식품     빙과류
96  20       식품     빙과류
97  20       식품     빙과류
98  20       식품     빙과류
99  20       식품     빙과류

[100 rows x 3 columns]


In [35]:
#1. data1에서 결제건수와 판매업종 추출
#2. data2에서 나이와 이용업종 추출 (판매업종=이용업종)
#3. 반복된 값 정리 -ing
#4. 판매량이 많은 연령 추출 
#   ex) 판매액, 판매건수 : (20,1위),(40,2위),(30,3위),(50,4위),(60,5위)
#5. 소비성향(k-means)
#   ex) 5개 업종 군집으로 구분

###판매량 순위 ...판매량이 높은 업종 정렬

#오름차순으로 정렬
data1_sorted_by_index01 = data1.sort_values(by='selng_cascnt' ,ascending=False)
data1_sorted_by_index02= data1.sort_values(by='salamt' ,ascending=False)

#가게-판매건수 리스트로 뽑기
sale_list = data1_sorted_by_index01[['mrhst_induty_cl_nm','selng_cascnt']].head(100)

sale_val = sale_list.values.tolist()

print(sale_val)

#가게-결제발생금액 리스트로 뽑기
earning_list = data1_sorted_by_index02[['mrhst_induty_cl_nm','salamt']].head(100)

earning_val = earning_list.values.tolist()

print(earning_val)

[['기타전문점', 5729589], ['기타전문점', 5015120], ['기타전문점', 4950263], ['기타전문점', 4832932], ['기타전문점', 4693220], ['기타전문점', 3534046], ['기타전문점', 3512725], ['기타전문점', 3354280], ['기타전문점', 3270602], ['기타전문점', 3130506], ['기타전문점', 2914198], ['기타전문점', 2291022], ['기타전문점', 2067345], ['기타전문점', 1798896], ['기타전문점', 1461410], ['기타전문점', 1349578], ['택   시', 1339996], ['택   시', 1324107], ['기타전문점', 1313331], ['기타전문점', 1174252], ['기타전문점', 1154537], ['기타전문점', 1141532], ['기타전문점', 1117068], ['기타전문점', 1081460], ['택   시', 1021016], ['택   시', 1008087], ['기타전문점', 983065], ['택   시', 978430], ['택   시', 954151], ['기타전문점', 950380], ['인터넷P/G', 922734], ['기타전문점', 918697], ['기타전문점', 912299], ['기타전문점', 912292], ['택   시', 907163], ['기타전문점', 884526], ['택   시', 868139], ['택   시', 867567], ['택   시', 864562], ['택   시', 860900], ['기타전문점', 854273], ['기타전문점', 852332], ['택   시', 837914], ['택   시', 826644], ['택   시', 811462], ['기타전문점', 808283], ['기타전문점', 803752], ['인터넷P/G', 795886], ['기타전문점', 795135], ['기타전문점', 788343], ['이동통신요금', 764941], [

In [38]:

###연령리스트, 각 행 비율....'해당업종' 이용연령대 추출
age_list=data2[['catm','age']]
age_val=age_list.values.tolist()

print(age_val)
data2['age'].value_counts(normalize=True)

###가게분류_list=data1['receipt_dttm']
#가게분류_val==가게분류_list.values.tolist()
#mrhst_induty_cl_nm 재분류(caltm '업종'카테고리에 맞게)
#비율

###소비성향...



[['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['건강관리용품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['가공식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], ['기호식품', '20'], [

all    0.167363
40     0.167026
50     0.166877
30     0.166736
20     0.166281
60     0.165717
Name: age, dtype: float64